In [1]:
# -*- coding: utf-8 -*-
import re
import json
import zipfile
from pathlib import Path

import pandas as pd

# ==== 경로 설정 ====
TL_ZIP = Path(r"C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\TL.zip")  # TL.zip 실제 위치로 수정
FOOD_CSV = Path(r"C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\food_ingredients_integration1.csv")  # CSV 위치로 수정
OUT_CSV = TL_ZIP.with_name("image_food_label_join_fast.csv")  # 결과 파일 (이전이랑 이름만 다르게)

# ==== 1) 파일 이름에서 음식명 추출 함수 ====
def extract_food_name(file_name: str) -> str:
    """
    예: A_13_A13001_가자미구이_30_09.jpg  ->  '가자미구이'
    """
    stem = file_name.rsplit(".", 1)[0]  # 확장자 제거
    parts = stem.split("_")
    for p in parts:
        if re.search(r"[가-힣]", p):
            return p
    return ""

# ==== 2) 식약처 CSV 불러오기 ====
food_df = pd.read_csv(FOOD_CSV)
print(f"CSV 식품명 개수: {len(food_df)}")

# ==== 3) ZIP 안 JSON 한 번 훑어서, 메타데이터 + unique 음식명 수집 ====
json_meta_list = []   # 각 JSON에 대한 {json_path, image_file, food_from_filename}
unique_food_names = set()

with zipfile.ZipFile(TL_ZIP) as z:
    json_names = [name for name in z.namelist() if name.endswith(".json")]
    total_json = len(json_names)
    print(f"ZIP 안 JSON 개수: {total_json}")

    for idx, name in enumerate(json_names, start=1):
        data = json.loads(z.read(name).decode("utf-8"))
        image_info = data["data"]["image_info"]
        image_file = image_info["file_name"]

        food_from_file = extract_food_name(image_file)

        json_meta_list.append({
            "json_path": name,
            "image_file": image_file,
            "food_from_filename": food_from_file,
        })

        if food_from_file:  # 빈 문자열은 제외
            unique_food_names.add(food_from_file)

        if idx % 1000 == 0 or idx == total_json:
            print(f"[1/3] JSON 메타 파싱 {idx}/{total_json} 완료")

print(f"서로 다른 음식 이름 개수: {len(unique_food_names)}")

# ==== 4) 음식 이름별로 한 번만 매칭해서 캐싱 ====
match_cache = {}  # food_name -> 매칭된 DataFrame (혹은 빈 DataFrame)

food_name_list = list(unique_food_names)
total_food_names = len(food_name_list)

for i, food_name in enumerate(food_name_list, start=1):
    # 1차: 완전 일치
    matches = food_df[food_df["식품명"] == food_name]

    # 2차: 부분 포함
    if len(matches) == 0:
        matches = food_df[food_df["식품명"].str.contains(food_name, na=False, regex=False)]

    match_cache[food_name] = matches  # 비어있을 수도 있음

    if i % 50 == 0 or i == total_food_names:
        print(f"[2/3] 음식명 매칭 {i}/{total_food_names} 완료")

# ==== 5) JSON 메타 + 캐시된 매칭 결과를 합쳐 최종 rows 생성 ====
rows = []

for idx, meta in enumerate(json_meta_list, start=1):
    food_from_file = meta["food_from_filename"]

    if not food_from_file or food_from_file not in match_cache:
        # 음식 이름이 없거나(한글 없음 등), 캐시에 없는 경우 → not_found
        rows.append({
            "json_path": meta["json_path"],
            "image_file": meta["image_file"],
            "food_from_filename": food_from_file,
            "식품코드": "",
            "식품명(B열)": "",
            "match_type": "not_found",
        })
    else:
        matches = match_cache[food_from_file]

        if matches is None or len(matches) == 0:
            # 캐시에 있지만, 실제 매칭 결과 없음
            rows.append({
                "json_path": meta["json_path"],
                "image_file": meta["image_file"],
                "food_from_filename": food_from_file,
                "식품코드": "",
                "식품명(B열)": "",
                "match_type": "not_found",
            })
        else:
            # 매칭된 후보 모두 기록 (여러 개일 수 있음)
            for _, row in matches.iterrows():
                rows.append({
                    "json_path": meta["json_path"],
                    "image_file": meta["image_file"],
                    "food_from_filename": food_from_file,
                    "식품코드": row["식품코드"],
                    "식품명(B열)": row["식품명"],
                    "match_type": "exact" if row["식품명"] == food_from_file else "contains",
                })

    if idx % 1000 == 0 or idx == len(json_meta_list):
        print(f"[3/3] 최종 row 생성 {idx}/{len(json_meta_list)} 완료")

# ==== 6) 결과 저장 ====
out_df = pd.DataFrame(rows)
out_df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print("완료! 결과 CSV:", OUT_CSV)


CSV 식품명 개수: 14584
ZIP 안 JSON 개수: 185553
[1/3] JSON 메타 파싱 1000/185553 완료
[1/3] JSON 메타 파싱 2000/185553 완료
[1/3] JSON 메타 파싱 3000/185553 완료
[1/3] JSON 메타 파싱 4000/185553 완료
[1/3] JSON 메타 파싱 5000/185553 완료
[1/3] JSON 메타 파싱 6000/185553 완료
[1/3] JSON 메타 파싱 7000/185553 완료
[1/3] JSON 메타 파싱 8000/185553 완료
[1/3] JSON 메타 파싱 9000/185553 완료
[1/3] JSON 메타 파싱 10000/185553 완료
[1/3] JSON 메타 파싱 11000/185553 완료
[1/3] JSON 메타 파싱 12000/185553 완료
[1/3] JSON 메타 파싱 13000/185553 완료
[1/3] JSON 메타 파싱 14000/185553 완료
[1/3] JSON 메타 파싱 15000/185553 완료
[1/3] JSON 메타 파싱 16000/185553 완료
[1/3] JSON 메타 파싱 17000/185553 완료
[1/3] JSON 메타 파싱 18000/185553 완료
[1/3] JSON 메타 파싱 19000/185553 완료
[1/3] JSON 메타 파싱 20000/185553 완료
[1/3] JSON 메타 파싱 21000/185553 완료
[1/3] JSON 메타 파싱 22000/185553 완료
[1/3] JSON 메타 파싱 23000/185553 완료
[1/3] JSON 메타 파싱 24000/185553 완료
[1/3] JSON 메타 파싱 25000/185553 완료
[1/3] JSON 메타 파싱 26000/185553 완료
[1/3] JSON 메타 파싱 27000/185553 완료
[1/3] JSON 메타 파싱 28000/185553 완료
[1/3] JSON 메타 파싱 29000/185553 완료
[1/3] JSON 메

In [1]:
# -*- coding: utf-8 -*-
import re
import json
import zipfile
from pathlib import Path

import pandas as pd

# ==== 경로 설정 ====
TL_ZIP = Path(r"C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\TL.zip")  # TL.zip 실제 위치로 수정
FOOD_CSV = Path(r"C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\food_ingredients_True.csv")  # CSV 위치로 수정
OUT_CSV = TL_ZIP.with_name("image_food_label_join_true.csv")  # 결과 파일 (이전이랑 이름만 다르게)

# ==== 1) 파일 이름에서 음식명 추출 함수 ====
def extract_food_name(file_name: str) -> str:
    """
    예: A_13_A13001_가자미구이_30_09.jpg  ->  '가자미구이'
    """
    stem = file_name.rsplit(".", 1)[0]  # 확장자 제거
    parts = stem.split("_")
    for p in parts:
        if re.search(r"[가-힣]", p):
            return p
    return ""

# ==== 2) 식약처 CSV 불러오기 ====
food_df = pd.read_csv(FOOD_CSV)
print(f"CSV 식품명 개수: {len(food_df)}")

# ==== 3) ZIP 안 JSON 한 번 훑어서, 메타데이터 + unique 음식명 수집 ====
json_meta_list = []   # 각 JSON에 대한 {json_path, image_file, food_from_filename}
unique_food_names = set()

with zipfile.ZipFile(TL_ZIP) as z:
    json_names = [name for name in z.namelist() if name.endswith(".json")]
    total_json = len(json_names)
    print(f"ZIP 안 JSON 개수: {total_json}")

    for idx, name in enumerate(json_names, start=1):
        data = json.loads(z.read(name).decode("utf-8"))
        image_info = data["data"]["image_info"]
        image_file = image_info["file_name"]

        food_from_file = extract_food_name(image_file)

        json_meta_list.append({
            "json_path": name,
            "image_file": image_file,
            "food_from_filename": food_from_file,
        })

        if food_from_file:  # 빈 문자열은 제외
            unique_food_names.add(food_from_file)

        if idx % 1000 == 0 or idx == total_json:
            print(f"[1/3] JSON 메타 파싱 {idx}/{total_json} 완료")

print(f"서로 다른 음식 이름 개수: {len(unique_food_names)}")

# ==== 4) 음식 이름별로 한 번만 매칭해서 캐싱 ====
match_cache = {}  # food_name -> 매칭된 DataFrame (혹은 빈 DataFrame)

food_name_list = list(unique_food_names)
total_food_names = len(food_name_list)

for i, food_name in enumerate(food_name_list, start=1):
    # 1차: 완전 일치
    matches = food_df[food_df["식품명"] == food_name]

    # 2차: 부분 포함
    if len(matches) == 0:
        matches = food_df[food_df["식품명"].str.contains(food_name, na=False, regex=False)]

    match_cache[food_name] = matches  # 비어있을 수도 있음

    if i % 50 == 0 or i == total_food_names:
        print(f"[2/3] 음식명 매칭 {i}/{total_food_names} 완료")

# ==== 5) JSON 메타 + 캐시된 매칭 결과를 합쳐 최종 rows 생성 ====
rows = []

for idx, meta in enumerate(json_meta_list, start=1):
    food_from_file = meta["food_from_filename"]

    if not food_from_file or food_from_file not in match_cache:
        # 음식 이름이 없거나(한글 없음 등), 캐시에 없는 경우 → not_found
        rows.append({
            "json_path": meta["json_path"],
            "image_file": meta["image_file"],
            "food_from_filename": food_from_file,
            "식품코드": "",
            "식품명(B열)": "",
            "match_type": "not_found",
        })
    else:
        matches = match_cache[food_from_file]

        if matches is None or len(matches) == 0:
            # 캐시에 있지만, 실제 매칭 결과 없음
            rows.append({
                "json_path": meta["json_path"],
                "image_file": meta["image_file"],
                "food_from_filename": food_from_file,
                "식품코드": "",
                "식품명(B열)": "",
                "match_type": "not_found",
            })
        else:
            # 매칭된 후보 모두 기록 (여러 개일 수 있음)
            for _, row in matches.iterrows():
                rows.append({
                    "json_path": meta["json_path"],
                    "image_file": meta["image_file"],
                    "food_from_filename": food_from_file,
                    "식품코드": row["식품코드"],
                    "식품명(B열)": row["식품명"],
                    "match_type": "exact" if row["식품명"] == food_from_file else "contains",
                })

    if idx % 1000 == 0 or idx == len(json_meta_list):
        print(f"[3/3] 최종 row 생성 {idx}/{len(json_meta_list)} 완료")

# ==== 6) 결과 저장 ====
out_df = pd.DataFrame(rows)
out_df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print("완료! 결과 CSV:", OUT_CSV)


CSV 식품명 개수: 7615
ZIP 안 JSON 개수: 185553
[1/3] JSON 메타 파싱 1000/185553 완료
[1/3] JSON 메타 파싱 2000/185553 완료
[1/3] JSON 메타 파싱 3000/185553 완료
[1/3] JSON 메타 파싱 4000/185553 완료
[1/3] JSON 메타 파싱 5000/185553 완료
[1/3] JSON 메타 파싱 6000/185553 완료
[1/3] JSON 메타 파싱 7000/185553 완료
[1/3] JSON 메타 파싱 8000/185553 완료
[1/3] JSON 메타 파싱 9000/185553 완료
[1/3] JSON 메타 파싱 10000/185553 완료
[1/3] JSON 메타 파싱 11000/185553 완료
[1/3] JSON 메타 파싱 12000/185553 완료
[1/3] JSON 메타 파싱 13000/185553 완료
[1/3] JSON 메타 파싱 14000/185553 완료
[1/3] JSON 메타 파싱 15000/185553 완료
[1/3] JSON 메타 파싱 16000/185553 완료
[1/3] JSON 메타 파싱 17000/185553 완료
[1/3] JSON 메타 파싱 18000/185553 완료
[1/3] JSON 메타 파싱 19000/185553 완료
[1/3] JSON 메타 파싱 20000/185553 완료
[1/3] JSON 메타 파싱 21000/185553 완료
[1/3] JSON 메타 파싱 22000/185553 완료
[1/3] JSON 메타 파싱 23000/185553 완료
[1/3] JSON 메타 파싱 24000/185553 완료
[1/3] JSON 메타 파싱 25000/185553 완료
[1/3] JSON 메타 파싱 26000/185553 완료
[1/3] JSON 메타 파싱 27000/185553 완료
[1/3] JSON 메타 파싱 28000/185553 완료
[1/3] JSON 메타 파싱 29000/185553 완료
[1/3] JSON 메타

In [2]:
import pandas as pd

# ==== 1. 원본 라벨 매칭 CSV 불러오기 ====
# fast 버전이든 원본이든 상관 없음. 파일 이름만 맞춰줘!
SRC_CSV = "image_food_label_join_true.csv"  # or "image_food_label_join.csv"

df = pd.read_csv(SRC_CSV)

print("원본 행 개수:", len(df))

# 혹시 match_type에 NaN이 있으면 not_found로 처리
df["match_type"] = df["match_type"].fillna("not_found")

# food_from_filename이 비어있거나 NaN인 건 일단 제외 (원하면 포함하도록 바꿀 수 있음)
df = df[df["food_from_filename"].notna() & (df["food_from_filename"] != "")]

print("유효한 food_from_filename이 있는 행 개수:", len(df))

# ==== 2. 음식별 이미지 개수도 같이 보고 싶으면 (옵션이지만 유용해서 넣어둠) ====
df["n_images_for_food"] = df.groupby("food_from_filename")["image_file"].transform("nunique")

# ==== 3. match_type 우선순위 부여 (exact > contains > not_found) ====
priority = {"exact": 0, "contains": 1, "not_found": 2}
df["match_priority"] = df["match_type"].map(priority)

# 혹시 이상한 값이 있으면 맨 뒤로 가도록 높은 숫자 부여
df["match_priority"] = df["match_priority"].fillna(99)

# ==== 4. 음식 이름 기준 + 우선순위 기준으로 정렬 후, 음식당 1행만 남기기 ====
df_sorted = df.sort_values(["food_from_filename", "match_priority"])

oneimage = df_sorted.drop_duplicates(subset=["food_from_filename"], keep="first")

print("음식 종류 수 (oneimage 행 수):", len(oneimage))

# 이제 oneimage에는
# - food_from_filename
# - 그 음식에 대해 가장 좋은(match_priority가 낮은) match_type
# - 대표 식품명(B열), 식품코드, 이미지 개수 등
# 이 1행만 남아 있음

# ==== 5. helper 컬럼 정리 ====
oneimage = oneimage.drop(columns=["match_priority"])

# ==== 6. 결과 간단 통계 찍어보기 ====
print("\n음식 기준 match_type 개수:")
print(oneimage["match_type"].value_counts())

# ==== 7. CSV로 저장 ====
OUT_CSV = "oneimage_food_label.csv"
oneimage.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print("\n저장 완료 👉", OUT_CSV)


원본 행 개수: 310215
유효한 food_from_filename이 있는 행 개수: 310215
음식 종류 수 (oneimage 행 수): 800

음식 기준 match_type 개수:
match_type
not_found    690
contains      99
exact         11
Name: count, dtype: int64

저장 완료 👉 oneimage_food_label.csv


In [3]:
import pandas as pd

df = pd.read_csv("oneimage_food_label.csv")  # 또는 image_food_label_join_fast.csv

valid = df[df["match_type"].isin(["exact", "contains"])]

unique_names = valid["식품명(B열)"].dropna().unique()

print(len(unique_names))
unique_names


108


array(['스파게티_간편조리세트_감바스알리오올리오', '감자그라탕', '감자빵_강원도알감자빵 (5개입)',
       '핫도그_통소시지감자핫도그', '계란빵', '라떼_고구마라떼 아이스(ICED)', '케이크_우유 고구마케이크',
       '김치찜_돼지고기_두부', '기타빵_까눌레', '바게트_스위트깜빠뉴', '깨찰빵_깨찰빵', '도넛_고구마품은 꽈배기',
       '롤빵_녹차 도쿄롤 (L)', '팥빵_단팥빵', '크림빵_단팥크림빵', '샌드위치_닭가슴살',
       '커피_더치커피 핫(HOT)', '도넛_딸기마카롱 도넛', '와플_딸기케이크 와플', '케이크_딸기크레이프 케이크',
       '떡강정_매콤치즈 소떡소떡', '식빵_러스크', '마늘빵', '머핀_마들렌', '마카롱_마약옥수수 뚱카롱',
       '소보로빵_맘모스빵', '버거_골든맥앤치즈비프 버거', '크림빵_골든크림메론빵', '번_모카번',
       '기타음료_라임 모히또', '라멘_미소라멘', '라떼_귀리 미숫가루 라떼', '스무디_바닐라밀크쉐이크 (R)',
       '라떼_바닐라라떼 아이스(ICED)', '식빵_뜯어먹는밤식빵 (H)', '버터크림빵', '케이크_너트 브라우니',
       '파이/만주_빨미까레 녹차', '케이크_생크림케이크', '버터빵_고메소금빵', '소보로빵_소보루빵',
       '소세지빵_든든한통소시지빵', '크림빵_슈크림빵', '롤빵_달콤한크림치즈시나몬롤', '찰떡빵_쑥떡쑥떡콩쑥이',
       '아이스티_디카페인 아샷추 아이스티', '코코아_아이스초코 아이스(ICED)', '아이스크림_리코타치즈 아포가토',
       '파이/만주_미니애플파이', '기타빵_얼그레이 까눌레', '케이크_투썸얼그레이밀크티쉬폰 케이크 (조각)',
       '타르트_소프트 에그타르트', '아이스크림_에스프레소 (더블)', '크림빵_후레쉬연유크림빵',
       '기타빵_우리쌀 막걸리로 만든 초당옥수수빵', '소보로빵_소보루 완두앙금빵', '크림빵_순진